In [ ]:
from monitorcontrol import get_monitors

monitors = get_monitors()
if not monitors:
    print("No monitors found")
    exit()

monitor = monitors[0]

Find the monitor:

In [ ]:
print("Attempting to find monitors...")

try:
    monitors = get_monitors()
    print(f"get_monitors() returned: {monitors}")
    print(f"Number of monitors: {len(monitors)}")
    
    if not monitors:
        print("\nNo monitors found via monitorcontrol")
        print("This might mean:")
        print("  - DDC/CI needs USB upstream cable connected")
        print("  - Monitor needs to be primary display")
        print("  - Windows blocking access")
    else:
        print(f"\nFound {len(monitors)} monitor(s), attempting access...")
        monitor = monitors[0]
        print(f"Monitor object: {monitor}")
        
        with monitor:
            print("Opened monitor context...")
            brightness = monitor.get_luminance()
            print(f"✓ Success! Current brightness: {brightness}")
            
except Exception as e:
    print(f"\n✗ Error occurred:")
    print(f"Type: {type(e).__name__}")
    print(f"Message: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Standard VCP codes to test
vcp_codes = {
    'brightness': 0x10,
    'contrast': 0x12,
    'red_gain': 0x16,
    'green_gain': 0x18,
    'blue_gain': 0x1A,
    'color_temp_increment': 0x8A,
    'color_preset': 0x14,
    'sharpness': 0x87,
    'input_source': 0x60,
    'power_mode': 0xD6,
    'display_application': 0xDC,
}

print("=== Testing Standard VCP Codes ===\n")

supported = {}

with monitor:
    for name, code in sorted(vcp_codes.items(), key=lambda x: x[1]):
        try:
            current, maximum = monitor.vcp.get_vcp_feature(code)
            print(f"✓ 0x{code:02X} {name:25s}: current={current:3d}, max={maximum:3d}")
            supported[name] = {'code': code, 'current': current, 'max': maximum}
        except Exception as e:
            print(f"✗ 0x{code:02X} {name:25s}: {type(e).__name__}")

print(f"\n=== Probing Manufacturer Codes (0xE0-0xFF) ===")
print("(Searching for BenQ-specific features like local dimming, black eQualizer)")
print("This may take 30-60 seconds...\n")

manufacturer_codes = {}

with monitor:
    for code in range(0xE0, 0x100):
        try:
            current, maximum = monitor.vcp.get_vcp_feature(code)
            print(f"✓ 0x{code:02X} manufacturer code: current={current}, max={maximum}")
            manufacturer_codes[f"0x{code:02X}"] = {'code': code, 'current': current, 'max': maximum}
        except:
            pass  # Skip unsupported codes silently

if manufacturer_codes:
    print(f"\n✓ Found {len(manufacturer_codes)} manufacturer-specific codes!")
else:
    print("\n✗ No manufacturer codes found in 0xE0-0xFF range")

# Also check some other ranges where manufacturers sometimes put custom codes
print("\n=== Checking Additional Ranges ===\n")

additional_ranges = [
    (0x70, 0x80, "Extended standard range"),
    (0xA0, 0xC0, "Manufacturer extended"),
]

with monitor:
    for start, end, description in additional_ranges:
        print(f"Scanning {description} (0x{start:02X}-0x{end:02X})...")
        for code in range(start, end):
            try:
                current, maximum = monitor.vcp.get_vcp_feature(code)
                print(f"  ✓ 0x{code:02X}: current={current}, max={maximum}")
                manufacturer_codes[f"0x{code:02X}"] = {'code': code, 'current': current, 'max': maximum}
            except:
                pass

# Save results
import json
import os

results = {
    'monitor': 'BenQ EX321UX',
    'standard_codes': supported,
    'manufacturer_codes': manufacturer_codes,
}

print("\n=== Summary ===")
print(f"Standard codes found: {len(supported)}")
print(f"Manufacturer codes found: {len(manufacturer_codes)}")

# Create data directory if it doesn't exist
os.makedirs('data', exist_ok=True)

with open('data/vcp_codes.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\n✓ Results saved to data/vcp_codes.json")

In [ ]:
from monitorcontrol import get_monitors
import time

monitors = get_monitors()
monitor = monitors[0]

def quick_test(code, values_to_test):
    """Quickly cycle through values"""
    with monitor:
        original, maximum = monitor.vcp.get_vcp_feature(code)
        print(f"\n0x{code:02X} - Original: {original}/{maximum}")
        
        for val in values_to_test:
            if val <= maximum:
                print(f"  Setting to {val}...", end='', flush=True)
                monitor.vcp.set_vcp_feature(code, val)
                time.sleep(1.5)  # Give you time to observe
                print(" [observe now]")
        
        # Restore
        monitor.vcp.set_vcp_feature(code, original)
        print(f"  Restored to {original}")

print("Watch for changes in:")
print("- Shadow detail (dark areas getting brighter)")
print("- Edge sharpness / motion blur")
print("- Color shifts")
print("- Zones dimming/brightening")
print("- Crosshair appearing\n")

# Test the promising codes
print("=== 0xFE Test (suspected Black eQ) ===")
quick_test(0xFE, [0, 10, 20])  # Low, medium, high

print("\n=== 0xFF Test (another 0-20 range) ===")
quick_test(0xFF, [0, 10, 20])

print("\n=== 0xED Test (0-6, suspected local dimming) ===")
quick_test(0xED, [0, 3, 6])

print("\n=== 0xF0 Test (1-3, suspected response time) ===")
quick_test(0xF0, [1, 2, 3])

print("\n=== 0xEA Test (binary toggle) ===")
quick_test(0xEA, [0, 1])

print("\n=== 0xF6 Test (binary toggle) ===")
quick_test(0xF6, [0, 1])

print("\nDone! All codes restored to original values.")

AVOID 0xF6! Kills display from ddc lockup.

In [ ]:
with monitor:
    # Check if brightness VALUE changes or just appears brighter
    print("Testing 0xF6 brightness effect...")
    
    # Get baseline brightness
    brightness_before, _ = monitor.vcp.get_vcp_feature(0x10)
    print(f"Brightness VCP before: {brightness_before}")
    
    # Toggle 0xF6 to 1 (brighter)
    monitor.vcp.set_vcp_feature(0xF6, 1)
    
    # Check brightness VCP again
    brightness_after, _ = monitor.vcp.get_vcp_feature(0x10)
    print(f"Brightness VCP after: {brightness_after}")
    
    if brightness_before == brightness_after:
        print("→ 0x10 didn't change, so 0xF6 is a MULTIPLIER or MODE")
        print("   (likely B.I.+, eco mode, or brightness boost)")
    else:
        print(f"→ 0x10 changed from {brightness_before} to {brightness_after}")
    
    # Restore
    monitor.vcp.set_vcp_feature(0xF6, 0)

In [ ]:
from monitorcontrol import get_monitors
import time

monitors = get_monitors()
monitor = monitors[0]

# AVOID: 0xF6 (breaks DDC)
# SAFE BETS: Codes with reasonable ranges that won't lock us out

safe_candidates = {
    '0xED (0-6, maybe local dimming)': 0xED,
    '0xFE (0-20, causes pink shift)': 0xFE,
    '0xFF (0-20, unknown)': 0xFF,
    '0xF0 (1-3, maybe response time)': 0xF0,
    '0xB6 (3-5, unknown)': 0xB6,
    '0xEA (0-1, toggle something)': 0xEA,  # Risky but might be useful
}

print("Testing codes one at a time...")
print("Watch for: dimming zones, shadow detail, motion blur, crosshair\n")

with monitor:
    for desc, code in safe_candidates.items():
        try:
            original, maximum = monitor.vcp.get_vcp_feature(code)
            print(f"\n=== {desc} ===")
            print(f"Current: {original}/{maximum}")
            
            # Test middle and max (avoid extremes on binary toggles)
            if maximum == 1:
                # Binary toggle - just flip it once
                print(f"Toggling to {1 - original}...")
                monitor.vcp.set_vcp_feature(code, 1 - original)
                time.sleep(2)
                print("What changed?")
                input("Press Enter to restore...")
                monitor.vcp.set_vcp_feature(code, original)
                
            else:
                # Range - test max only
                print(f"Setting to MAX ({maximum})...")
                monitor.vcp.set_vcp_feature(code, maximum)
                time.sleep(2)
                print("What changed?")
                input("Press Enter to restore...")
                monitor.vcp.set_vcp_feature(code, original)
                
            print("✓ Restored")
            
        except Exception as e:
            print(f"✗ Error testing {desc}: {e}")
            print("Skipping to next code...")

print("\n✓ All tests complete")

In [ ]:
BENQ_VCP_CODES = {
    # Standard codes
    'brightness': 0x10,
    'contrast': 0x12,
    'red_gain': 0x16,
    'green_gain': 0x18,
    'blue_gain': 0x1A,
    'sharpness': 0x87,
    
    # BenQ-specific
    'local_dimming': 0xEA,  # 0=Off, 1=On (binary toggle)
    
    # Unknown/untested
    'mystery_pink_shift': 0xFE,  # 0-20, causes color shift
    'unknown_0xFF': 0xFF,  # 0-20
    'unknown_0xED': 0xED,  # 0-6
    'unknown_0xF0': 0xF0,  # 1-3
    
    # DANGEROUS - DO NOT USE
    'DO_NOT_TOUCH_0xF6': 0xF6,  # Breaks DDC
}

In [4]:
from monitorcontrol import get_monitors
import time

monitors = get_monitors()
monitor = monitors[0]

with monitor:
    print("=== Testing Local Dimming System ===\n")
    
    # Step 1: Turn local dimming ON
    print("Step 1: Enabling local dimming (0xEA = 1)...")
    monitor.vcp.set_vcp_feature(0xEA, 1)
    time.sleep(1)
    print("✓ Local dimming should now be ON\n")
    
    # Step 2: Now test intensity codes
    print("Step 2: Testing 0xED (0-6) for intensity levels...")
    print("Watch for dimming zones getting more/less aggressive\n")
    
    original_ed, _ = monitor.vcp.get_vcp_feature(0xED)
    print(f"Original 0xED value: {original_ed}")
    
    for val in [0, 2, 4, 6]:
        print(f"  Setting 0xED to {val}...", end='', flush=True)
        monitor.vcp.set_vcp_feature(0xED, val)
        time.sleep(2)
        print(" [observe dimming zones]")
    
    input("\nPress Enter to continue...")
    
    # Restore
    monitor.vcp.set_vcp_feature(0xED, original_ed)
    print(f"✓ Restored 0xED to {original_ed}\n")
    
    # Step 3: Test other candidates while dimming is ON
    print("Step 3: Testing 0xF0 (1-3) with local dimming ON...")
    
    original_f0, _ = monitor.vcp.get_vcp_feature(0xF0)
    print(f"Original 0xF0 value: {original_f0}")
    
    for val in [1, 2, 3]:
        print(f"  Setting 0xF0 to {val}...", end='', flush=True)
        monitor.vcp.set_vcp_feature(0xF0, val)
        time.sleep(2)
        print(" [observe]")
    
    monitor.vcp.set_vcp_feature(0xF0, original_f0)
    print(f"✓ Restored 0xF0 to {original_f0}\n")
    
    # Step 4: Turn local dimming back to original state
    print("Step 4: Checking original local dimming state...")
    # You might have had it off originally
    original_ea, _ = monitor.vcp.get_vcp_feature(0xEA)
    
    restore_ea = input(f"Restore local dimming to OFF (0) or ON (1)? Current read as {original_ea}: ")
    if restore_ea in ['0', '1']:
        monitor.vcp.set_vcp_feature(0xEA, int(restore_ea))
        print(f"✓ Set to {restore_ea}")

print("\n=== Test Complete ===")

=== Testing Local Dimming System ===

Step 1: Enabling local dimming (0xEA = 1)...
✓ Local dimming should now be ON

Step 2: Testing 0xED (0-6) for intensity levels...
Watch for dimming zones getting more/less aggressive

Original 0xED value: 0
  Setting 0xED to 0... [observe dimming zones]
  Setting 0xED to 2... [observe dimming zones]
  Setting 0xED to 4... [observe dimming zones]
  Setting 0xED to 6... [observe dimming zones]
✓ Restored 0xED to 0

Step 3: Testing 0xF0 (1-3) with local dimming ON...


VCPError: failed to get VCP feature: An error occurred because the checksum field in a DDC/CI message did not match the message's computed checksum value. This error implies that the data was corrupted while it was being transmitted from a monitor to a computer.

In [ ]:
# data/vcp_codes.json
{
  "brightness": 0x10,      # 0-100
  "blue_gain": 0x1A,       # 0-100 (for night mode)
  "local_dimming": 0xEA,   # 0=Off, 1=On
  "sharpness": 0x87,       # 0-10
}